In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/kyleomalley/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [3]:
# Page to be scraped - Mar's News
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [4]:
title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find('div', class_='article_teaser_body').text


print(title)
print('----------')
print(news_p)


MOXIE Could Help Future Rockets Launch Off Mars
----------
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [5]:
# Mars image scraping
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(image_url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Featured image link
find_image_url = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

featured_image_url = image_url + find_image_url
                                                           
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA17440-1920x1200.jpg'

In [7]:
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

read_table = pd.read_html(facts_url)
read_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [8]:
mars_facts = read_table[0]
mars_facts.columns = ['Description', 'Value']

facts_html_table = mars_facts.to_html(index= False, justify='left')
facts_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: left;">\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [9]:
facts_html_table.replace('\n', '')
print(facts_html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: left;">
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemisphere

In [10]:
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)

In [11]:
# create loop to find image urls
html = browser.html

soup = BeautifulSoup(html, 'html.parser')

hemis = soup.find_all('div', class_='item')

hemis_image_urls = []

main_url = 'https://astrogeology.usgs.gov'

# loop through items for images
for img in hemis:
    title = img.find('h3').text
    
    # need link to full res image
    specific_url = img.find('a', class_='itemLink product-item')['href']
    
    # visit link for image
    browser.visit(main_url + specific_url)
    
    # Now get html for page to find src to add to url list
    specific_url_html = browser.html
    
    soup = BeautifulSoup(specific_url_html, 'html.parser')
    
    imageurl = main_url + soup.find('img', class_='wide-image')['src']
    
    hemis_image_urls.append({"title" : title, "img_url" : imageurl})
    
hemis_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]